In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import os,cv2,glob

TensorFlow 2.x selected.


In [0]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [0]:
def image_example(image_string,label):
    feature = {
      'image_raw': _bytes_feature(image_string),
        'label': _float_feature(label),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))  

In [0]:
class_list = ['COVID-19','Normal']

In [0]:
train_dataset = tf.data.TFRecordDataset(glob.glob('/content/drive/My Drive/COVID-19-Xrays/Binary/Train/*.tfrecord'),num_parallel_reads=3)
val_dataset = tf.data.TFRecordDataset(glob.glob('/content/drive/My Drive/COVID-19-Xrays/Binary/Test/*.tfrecord'),num_parallel_reads=3)

image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([2],dtype=tf.float32)
}

def _parse_image_function(example_proto):
  features = tf.io.parse_single_example(example_proto, image_feature_description)
  image_raw = tf.image.decode_jpeg(features['image_raw'],channels=1)
  image_raw = tf.cast(image_raw , tf.float32) * (1. / 255)
  image_raw = tf.image.resize(image_raw,(300,400))

  label = features['label']

  return image_raw,label

train_dataset = train_dataset.map(_parse_image_function)
val_dataset = val_dataset.map(_parse_image_function)

In [7]:
c0,c1 = 0,0
for i,j in train_dataset:
  if (j.numpy()==[1,0]).all():
    c0+=1
  elif (j.numpy()==[0,1]).all():
    c1+=1
c0,c1    

(180, 1781)

In [8]:
c2,c3 = 0,0
for i,j in val_dataset:
  if (j.numpy()==[1,0]).all():
    c2+=1
  elif (j.numpy()==[0,1]).all():
    c3+=1
c2,c3      

(9, 450)

In [9]:
TRAIN_BATCH = 16
VAL_BATCH = 16

train_dataset = train_dataset.repeat().shuffle(2000).batch(TRAIN_BATCH).prefetch(8)
val_dataset = val_dataset.repeat().shuffle(500).batch(VAL_BATCH)
train_dataset,val_dataset

(<PrefetchDataset shapes: ((None, 300, 400, 1), (None, 2)), types: (tf.float32, tf.float32)>,
 <BatchDataset shapes: ((None, 300, 400, 1), (None, 2)), types: (tf.float32, tf.float32)>)

In [0]:
distribution = {1:1781 , 0: 180}
total_train_samples = 180+1781
total_test_samples = 450+9

In [11]:
samples = {0:(1781+180)/(2*180),1:(1781+180)/(2*1781)}
print(samples)

{0: 5.447222222222222, 1: 0.5505334081976417}


In [0]:
from tensorflow.keras.layers import Dense,Conv2D,SeparableConv2D,Dropout,MaxPool2D,MaxPooling2D,GlobalMaxPooling2D,BatchNormalization,Flatten,Dense

def create_model(shape=(300,400,1)):
    model = tf.keras.models.Sequential()
    
    model.add(Conv2D(512,5,(2,2),activation='relu',input_shape=shape))
    model.add(Conv2D(256,2,activation='relu'))
    model.add(MaxPool2D())
  
    model.add(Conv2D(32,3,activation='relu'))
    model.add(Conv2D(128,1,activation='relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(32,4,(2,2),activation='relu'))
    model.add(Conv2D(64,2))
    model.add(MaxPool2D())

    model.add(GlobalMaxPooling2D())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(len(class_list),activation='softmax'))            

    return model

In [13]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 198, 512)     13312     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 197, 256)     524544    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 98, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 96, 32)        73760     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 71, 96, 128)       4224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 48, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 23, 32)        6

In [0]:
optimizer = tf.keras.optimizers.SGD(momentum=0.9,learning_rate=0.005)
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=[],weighted_metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC()])

In [0]:
tf.keras.backend.clear_session()

In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/COVID-19-Xrays/22_03_2020.h5')

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,verbose=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('Checkpoint.h5',save_best_only=True,verbose=1)

In [16]:
history = model.fit(train_dataset,validation_data=val_dataset,shuffle=True,epochs=40,class_weight=samples,steps_per_epoch=total_train_samples//TRAIN_BATCH,validation_steps=total_test_samples//VAL_BATCH,callbacks=[early_stopping,model_checkpoint])

Train for 122 steps, validate for 28 steps
Epoch 1/40
121/122 [============================>.] - ETA: 0s - loss: 0.7545 - accuracy: 0.5598 - precision: 0.5598 - recall: 0.5598 - auc: 0.5745
Epoch 00001: val_loss improved from inf to 0.45156, saving model to Checkpoint.h5
122/122 [==============================] - 48s 390ms/step - loss: 0.7549 - accuracy: 0.5600 - precision: 0.5600 - recall: 0.5600 - auc: 0.5744 - val_loss: 0.4516 - val_accuracy: 0.7151 - val_precision: 0.7151 - val_recall: 0.7151 - val_auc: 0.8255
Epoch 2/40
121/122 [============================>.] - ETA: 0s - loss: 0.5537 - accuracy: 0.7488 - precision: 0.7488 - recall: 0.7488 - auc: 0.8040
Epoch 00002: val_loss improved from 0.45156 to 0.33437, saving model to Checkpoint.h5
122/122 [==============================] - 38s 310ms/step - loss: 0.5545 - accuracy: 0.7490 - precision: 0.7490 - recall: 0.7490 - auc: 0.8045 - val_loss: 0.3344 - val_accuracy: 0.8106 - val_precision: 0.8106 - val_recall: 0.8106 - val_auc: 0.8538

In [0]:
model = tf.keras.models.load_model('Checkpoint.h5')

In [0]:
model = tf.keras.models.load_model('/content/drive/My Drive/COVID-19-Xrays/Binary/BinaryCOVID-19Classifier.h5')

In [0]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
def metrics(list1,list2,roc_list):
  accuracy = balanced_accuracy_score(list1,list2)
  roc_auc = roc_auc_score(list1,roc_list,multi_class='ovr',labels=[0,1],average='weighted')
  precision,recall,fscore,_ = precision_recall_fscore_support(list1,list2,labels=[0,1],average='weighted')
  return accuracy,precision,recall,fscore,roc_auc

In [37]:
image,files,roc_list = [],[],[]
acc=0
for i in glob.glob('/content/drive/My Drive/COVID-19-Xrays/Binary/test/*.jpeg')+glob.glob('/content/drive/My Drive/COVID-19-Xrays/Binary/test/*.png')+glob.glob('/content/drive/My Drive/COVID-19-Xrays/Binary/test/*.jpg'):
  img = open(i,'rb').read()
  img = tf.io.decode_jpeg(img,channels=1)
  img = tf.cast(img , tf.float32) * (1. / 255)
  img = tf.image.resize(img,(300,400))
  preds = model.predict(np.expand_dims(img,axis=0))
  roc_list.append(preds[0])
  image.append(np.argmax(preds))
  name = os.path.basename(i).split('.')[0]
  if name.find('NORMAL')!=-1:
    files.append(1) 
  else:
    files.append(0)   

accuracy,precision,recall,fscore,auc = metrics(files,image,image)
print("Accuracy on Test Set is: ",accuracy)
print("Precision on Test Set is: ", precision)
print("Recall on Test Set is: ",recall)
print("Fscore: ",fscore)
print("AUC Score: ",auc)


Accuracy on Test Set is:  0.9621318373071529
Precision on Test Set is:  0.9629629629629629
Recall on Test Set is:  0.9629629629629629
Fscore:  0.9629629629629629
AUC Score:  0.9621318373071529


In [0]:
# model.save('/content/drive/My Drive/COVID-19-Xrays/Binary/BinaryCOVID-19Classifier.h5')

In [32]:
image,files = [],[]
acc=0
for i in glob.glob('/content/*.jpeg')+glob.glob('/content/*.png')+glob.glob('/content/*.jpg'):
  img = open(i,'rb').read()
  img = tf.io.decode_jpeg(img,channels=1)
  img = tf.cast(img , tf.float32) * (1. / 255)
  img = tf.image.resize(img,(300,400))
  preds = model.predict_classes(np.expand_dims(img,axis=0))
  image.append(class_list[preds[0]])
  name = os.path.basename(i).split('.')[0]
  files.append(name)
for i,j in zip(files,image):
  print(i,j)

# Got one wrong :P

5e6dd879fde9502400e58b2f COVID-19
171CB377-62FF-4B76-906C-F3787A01CB2E COVID-19
FE9F9A5D-2830-46F9-851B-1FF4534959BE Normal
C6EA0BE5-B01E-4113-B194-18D956675E25 COVID-19
5931B64A-7B97-485D-BE60-3F1EA76BC4F0 COVID-19
03BF7561-A9BA-4C3C-B8A0-D3E585F73F3C COVID-19
7EF28E12-F628-4BEC-A8C5-E6277C2E4F60 COVID-19
